In [1]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z','z1','z2','z3','z4','z5']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l','l1','l2','l3','l4','l5']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)

x,y,zz,z1,z2,z3,z4,z5=KT_ring.gens()
t,m,ll,l1,l2,l3,l4,l5= coho_ring.gens()
b=var('b')
W=var('w')
V=var('v')
L=var('l')
z=[z1,z2,z3,z4,z5]



def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom

def total_chern(f):
    numer, denom = from_character(f)
    new_numer=[i+1 for i in numer]
    new_denom=[i+1 for i in denom]
    return coho_ring.fraction_field()(prod(new_numer+[1]) / (prod(new_denom)))


def total_chern_vb(p):
    Q = character(p,x,y)
    roots = []
    vb_weight = W
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        roots += (coeff) * [wt+vb_weight]
    total = prod((1+r) for r in roots)
    return total

def total_segre_vb(p):
    Q = character(p,x,y)
    roots = []
    vb_weight = W
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        roots += (coeff) * [wt+vb_weight]
    total = prod(1/(1+r) for r in roots)
    return total

def determinant_vb(p,n):
    Q = character(p,x,y)
    det = determinant(Q)*(exp(W*n))
    return det

def determinant_vb_rank_2(p,n):
    Q = character(p,x,y=1/x)
    det = (determinant(Q)^2)*(exp((W+V)*n))
    return det

def total_chern_vb_rank_2(p):
    Q = character(p,x,y)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        roots += (coeff) * [wt+W]
        roots += (coeff) * [wt+V]
    total = prod((1+r) for r in roots)
    return total

def Chern_Num(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_chern_vb(p)
        total+=add
    return total

def Chern_Num_rank_2(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_chern_vb_rank_2(p)
        total+=add
    return total

def Chern_Num_vir(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_chern_vb(p)*top_chern(dual(canonical_bundle(p,x,y)))
        total+=add
    return total

def Chern_Num_vir_rank_2(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_chern_vb_rank_2(p)*top_chern(dual(canonical_bundle(p,x,y)))
        total+=add
    return total

def Segre_Num(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_segre_vb(p)
        total+=add
    return total

def Segre_Num_vir(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*total_segre_vb(p)*top_chern(dual(canonical_bundle(p,x,y)))
        total+=add
    return total


def Verlinde_Num(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p))*\
            determinant_vb(p,n)
        total+=add
    return total



def Verlinde_Num_inv(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p))*\
            (1/determinant_vb(p,n))
        total+=add
    return total



def Verlinde_Num_vir(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p)-dual(canonical_bundle(p,x,y)))*\
            determinant_vb(p,n)
        total+=add
    return total

def Verlinde_Num_vir_rank_2(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p)-dual(canonical_bundle(p,x,y)))*\
            determinant_vb_rank_2(p,n)
        total+=add
    return total

def Verlinde_Num_vir_inv(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p))*\
            (1/determinant_vb(p,n))*\
            measure_unsymmetrized(-canonical_bundle(p,x,y),inv=False)
        total+=add
    return total

def measure_z_vb_1(p):
    f = character_vb_rank_1(p,x,y)
    L = f.expand().operands()
    if p.size()==1:
        L=[exp(W),]
    total=1
    for monomial in L:
        term = 1 - monomial*z
        total*=term
    return total

def custom_invariant(n):
    total=0
    for p in Partitions(n):
        add=measure_unsymmetrized(tangent(p)-dual(canonical_bundle(p,x,y)))*\
            determinant(dual(character_vb(p,(0,0),x,y)))*\
            measure_z_vb_1(p)
        total+=add
    return (-1)^n*total

def character_vb_rank_2(p,x=x,y=y):
    Q = character(p,x,y)
    return Q * (exp(W)+exp(V))

def character_vb_rank_1(p,x=x,y=y):
    Q = character(p,x,y)
    return Q * (exp(W))


def measure_z_vb(p):
    f = character_vb_rank_2(p,x,y)
    L = f.expand().operands()
    total=1
    for monomial in L:
        term = 1 - monomial*z
        total*=term
    return total

def Master_Partition_rank_2(n):
    total=0
    for p in Partitions(n):
        add=determinant(dual(character_vb(p,(0,0),x,y)))*\
            measure_unsymmetrized(tangent(p))*\
            measure_z_vb(p)
        total+=add
    return (-1)^n*total

def DT_Num(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))
        total+=add
    return total

def DT_Num_vir(n):
    total=0
    for p in Partitions(n):
        add=top_chern(-tangent(p))*top_chern(dual(canonical_bundle(p,x,y)))
        total+=add
    return total

def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f

In [9]:
p=Partition([3,1,1])
tangent(p,x,1/x)-dual(canonical_bundle(p,x,1/x))

x^5 + x^2 + x - 1 + x^-1 + x^-2 + x^-5

In [8]:
(1-character(p,x=x^-1,y=x)*(1-x)*(1-1/x))*character(p,x=x,y=x^-1)

x^5 + x^2 + x - 1 + x^-1 + x^-2 + x^-5

In [38]:
Partitions(5).list()

[[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]

In [2]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            total+=(1-character(Z[i],x=x^-1,y=x)*(1-x)*(1-1/x))*character(Z[j],x=x,y=x^-1)*(zz^(j-i))
    return total+1


def total_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        roots = []
        vb_weight = W
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            roots += (coeff) * [wt+vb_weight]
        total *= prod((1+r) for r in roots)
    return total

def Chern_Num_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot(Z)
        total+=add
    return total

def total_segre_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        roots = []
        vb_weight = W
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            roots += (coeff) * [wt+vb_weight]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z)
        total+=add
    return total


def determinant_quot(Z,n):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        det *= determinant(Q)
    return det*(exp(W*n))

def Verlinde_Num_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n)
        total+=add
    return total

def Verlinde_Num_quot_inv(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n))
        total+=add
    return total


def deg0(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v,l=b*l,l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5),0))

def degN(f,n):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v,l=b*l,l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5),n))


def deg1(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v,l=b*l,l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5),-1))

def total_chern_quot_2(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            roots += (coeff) * [wt+w]
            roots += (coeff) * [wt+V]
        total *= prod((1+r) for r in roots)
    return total

def Chern_Num_quot_2(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot_2(Z)
        total+=add
    return total


def total_segre_quot_2(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        roots = []
        vb_weight = W
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            roots += (coeff) * [wt+w]
            roots += (coeff) * [wt+V]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot_2(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot_2(Z)
        total+=add
    return total

def determinant_quot_2(Z,n):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(zz^i)
        det *= determinant(Q)
    return det^2*(exp((W+V)*n))

def Verlinde_Num_quot_2(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot_2(Z,n)
        total+=add
    return total

def Verlinde_Num_quot_2_inv(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot_2(Z,n))
        total+=add
    return total

def all_check(start,finish):
    # check for n,N,r, up to r=5
    with open("symmetry2d-segre-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,4):
            for k in range(1,4):
                n,N,r=[i,j,k]
                if k<j:
                    continue
                if j<k:
                    print("checking segre sym: n="+str(n)+", N="+str(N)+", r="+str(r))
                    if str(n)+','+str(N)+','+str(r) in lines:
                        print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
                    else:
                        S1=Segre_Num_quot(n,N,r)
                        S2=Segre_Num_quot(n,r,N)
                        print("comparing")
                        diff=S1*(-1)^(n*N)-symmetrize(S2)*(-1)^(n*r)
                        if diff==0:
                            writeline("symmetry2d-segre-checked.txt",str(n)+','+str(N)+','+str(r))
                            print("checked")
                        else:
                            print("failed")
                              
        print("all checked for: n="+str(i))
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('V2d-red-checked.txt', 'w').close()

In [4]:
deg0(pretty(Segre_Num_quot(1, 1)))

-w^2/t^2

In [5]:
deg0(pretty(Segre_Num_quot(2, 1)))

1/2*(t^2 + 6*w^2)/t^2

In [6]:
deg0(pretty(Segre_Num_quot(3, 1)))

-10/3*(t^2 + 3*w^2)/t^2

In [9]:
pretty(Chern_Num_quot(1, 1))

-(w + 1)/t^2

In [10]:
pretty(Chern_Num_quot(2, 1))

-1/2*(w + 1)/t^2

In [11]:
pretty(Chern_Num_quot(1, 1))

-(w + 1)/t^2

In [7]:
fixed_locus(2,3)

[([], [], [2]),
 ([], [], [1, 1]),
 ([], [1], [1]),
 ([], [2], []),
 ([], [1, 1], []),
 ([1], [], [1]),
 ([1], [1], []),
 ([2], [], []),
 ([1, 1], [], [])]

In [64]:
deg1(pretty(Verlinde_Num_quot(1,2))(x=exp(t),y=exp(m),z1=exp(l1),z2=exp(l2)))

-(l1 + l2 + 2*w)/t^2

In [65]:
deg1(pretty(Segre_Num_quot(1, 2)))

-(l1 + l2 + 2*w)/t^2

In [30]:
deg1(pretty(Verlinde_Num_quot(1,3))(x=exp(t),y=exp(m),z=exp(l)))

-3*(l + w)/t^2

In [29]:
deg1(pretty(Segre_Num_quot(1, 3)))

3*(l + w)/t^2

In [19]:
deg1(pretty(Verlinde_Num_quot(2,1))(x=exp(t),y=exp(m),z=exp(l)))

-3/2*w/t^2

In [20]:
deg1(pretty(Segre_Num_quot(2, 1)))

-3/2*w/t^2

In [21]:
deg1(pretty(Verlinde_Num_quot(2,2))(x=exp(t),y=exp(m),z=exp(l)))

-5/2*(l + 2*w)/t^2

In [22]:
deg1(pretty(Segre_Num_quot(2, 2)))

-5/2*(l + 2*w)/t^2

In [32]:
deg1(pretty(Verlinde_Num_quot(2,3))(x=exp(t),y=exp(m),z=exp(l)))

-21/2*(l + w)/t^2

In [31]:
deg1(pretty(Segre_Num_quot(2, 3)))

-21/2*(l + w)/t^2

In [34]:
deg1(pretty(Verlinde_Num_quot(2,4))(x=exp(t),y=exp(m),z=exp(l)))

-9*(3*l + 2*w)/t^2

In [33]:
deg1(pretty(Segre_Num_quot(2, 4)))

-9*(3*l + 2*w)/t^2

In [36]:
deg1(pretty(Verlinde_Num_quot(3,1))(x=exp(t),y=exp(m),z=exp(l)))

-10/3*w/t^2

In [35]:
deg1(pretty(Segre_Num_quot(3, 1)))

10/3*w/t^2

In [37]:
deg1(pretty(Verlinde_Num_quot(3,2))(x=exp(t),y=exp(m),z=exp(l)))

-28/3*(l + 2*w)/t^2

In [43]:
deg1(pretty(Segre_Num_quot(3, 2)))

-28/3*(l + 2*w)/t^2

In [39]:
deg1(pretty(Verlinde_Num_quot(3,3))(x=exp(t),y=exp(m),z=exp(l)))

-55*(l + w)/t^2

In [44]:
deg1(pretty(Segre_Num_quot(3, 3)))

55*(l + w)/t^2

In [42]:
deg1(pretty(Verlinde_Num_quot(4,2))(x=exp(t),y=exp(m),z=exp(l)))

-165/4*(l + 2*w)/t^2

In [45]:
deg1(pretty(Segre_Num_quot(4, 2)))

-165/4*(l + 2*w)/t^2

In [46]:
deg1(pretty(Verlinde_Num_quot(4,3))(x=exp(t),y=exp(m),z=exp(l)))

-1365/4*(l + w)/t^2

In [47]:
deg1(pretty(Segre_Num_quot(4, 3)))

-1365/4*(l + w)/t^2

# RANK 2


In [53]:
deg1(pretty(Verlinde_Num_quot_2(2,2))(x=exp(t),y=exp(m),z=exp(l)))

-35/2*(l + v + w)/t^2

In [52]:
deg1(pretty(Segre_Num_quot_2(2, 2)))

-35/2*(l + v + w)/t^2

In [110]:
deg1(pretty(Verlinde_Num_quot_2(3,3))(x=exp(t),y=exp(m),z1=exp(l1),z2=exp(l2),z3=exp(l3)))

-1001/3*(2*l1 + 2*l2 + 2*l3 + 3*v + 3*w)/t^2

In [116]:
deg1(Segre_Num_quot_2(2, 3))

-21*(2*l1 + 2*l2 + 2*l3 + 3*v + 3*w)/t^2

In [256]:
degN(Segre_Num_quot_2(4, 3)*b^5,2)

0

# Degree 0

In [69]:
degn(Verlinde_Num_quot(3,1)(x=exp(t*b),z1=exp(l1*b),w=w*t*b),0)

-5/6*((6*w^2 + 1)*t^2 + 12*l1*t*w + 6*l1^2)/t^2

In [68]:
degn(Segre_Num_quot(3,1)(t=t*b,l1=l1*b,w=w*t*b),0)

-10/3*(3*t^2*w^2 + 6*l1*t*w + 3*l1^2 + t^2)/t^2

In [86]:
Segre_Num_quot(1,2)

-1/((l1*t^2 - l2*t^2)*(l1 + w + 1)) + 1/((l1*t^2 - l2*t^2)*(l2 + w + 1))

Note the non-unit part ($e(T^{\text{vir}})$) of the dernominator of each individual term contains $l_1,l_2$, but after summing up,

In [87]:
pretty(Segre_Num_quot(1,2))

1/((l1 + w + 1)*(l2 + w + 1)*t^2)

the non-unit part becomes $t^2$, and taking degree 0 we get something whose denominator is just $t^2$, independent of $l_1,l_2$. Namely we only get positive degrees in $l_1,l_2$, and the negative degrees vanish, as if the Quot scheme is compact. So the $\mathbb{C}^*$ action does not see the non-compactness of individual Hilbert space in the fixed locus?

In [91]:
pretty(degn(Segre_Num_quot(2,2)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

15*w^2 + 2

In [96]:
pretty(degn(Segre_Num_quot(3,2)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

84*w^2 + 22

In [95]:
pretty(degn(Segre_Num_quot(4,2)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

495*w^2 + 385/2

In [104]:
pretty(degn(Segre_Num_quot(5,2)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

3003*w^2 + 1547

In [100]:
pretty(degn(Segre_Num_quot(2,1)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

3*w^2 + 1/2

In [101]:
pretty(degn(Segre_Num_quot(3,1)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

-10*w^2 - 10/3

In [102]:
pretty(degn(Segre_Num_quot(4,1)(t=t*b,l1=l1*b,l2=l2*b,w=w*t*b),0)(l1=0,l2=0))

35*w^2 + 35/2

In [16]:
pretty(deg1(pretty(Verlinde_Num_quot(2,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-3/2*w/t

In [17]:
pretty(deg1(pretty(Verlinde_Num_quot(3,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-10/3*w/t

In [168]:
pretty(deg0(pretty(Verlinde_Num_quot(2,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-3/2*w^2

In [169]:
pretty(deg0(pretty(Verlinde_Num_quot(3,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-5*w^2 - 5/6

In [170]:
pretty(deg0(pretty(Verlinde_Num_quot(4,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-35/2*w^2 - 35/6

In [171]:
pretty(deg0(pretty(Verlinde_Num_quot(5,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-63*w^2 - 63/2

$N=1$, leading coeff is 
$$\frac14\binom{2n}{n}$$

In [155]:
pretty(deg0(pretty(Verlinde_Num_quot(2,2))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-5*w^2 + 1/12

In [153]:
pretty(deg0(pretty(Verlinde_Num_quot(3,2))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-28*w^2 - 10/3

In [156]:
pretty(deg0(pretty(Verlinde_Num_quot(4,2))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-165*w^2 - 165/4

In [157]:
pretty(deg0(pretty(Verlinde_Num_quot(5,2))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-1001*w^2 - 2275/6

$N=2$, leading coeff is 
$$\frac13\binom{3n}{n}$$

In [164]:
pretty(deg0(pretty(Verlinde_Num_quot(2,3))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-21/2*w^2 + 1/4

In [165]:
pretty(deg0(pretty(Verlinde_Num_quot(3,3))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-165/2*w^2 - 33/4

In [166]:
pretty(deg0(pretty(Verlinde_Num_quot(4,3))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-1365/2*w^2 - 595/4

In [167]:
pretty(deg0(pretty(Verlinde_Num_quot(5,3))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-5814*w^2 - 1938

$N=3$, leading coeff is 
$$\frac38\binom{4n}{n}$$

In [173]:
pretty(deg0(pretty(Verlinde_Num_quot(2,4))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-1/2*(6*w + 1)*(6*w - 1)

In [174]:
pretty(deg0(pretty(Verlinde_Num_quot(3,4))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-182*w^2 - 49/3

In [175]:
pretty(deg0(pretty(Verlinde_Num_quot(4,4))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-1938*w^2 - 779/2

In [176]:
pretty(deg0(pretty(Verlinde_Num_quot(5,4))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,l=0))

-21252*w^2 - 6578

$N=4$, leading coeff is 
$$\frac25\binom{5n}{n}$$

In [20]:
pretty(deg0(pretty(Verlinde_Num_quot_2_inv(2,1))(x=exp(t),z=exp(l)))(w=t*w,v=t*v,l=0))

3/2*(v + w)^2

In [21]:
pretty(deg0(pretty(Verlinde_Num_quot_2_inv(3,1))(x=exp(t),z=exp(l)))(w=t*w,v=t*v,l=0))

-5*v^2 - 10*v*w - 5*w^2 - 5/6

In [228]:
pretty(deg0(pretty(Verlinde_Num_quot_2_inv(4,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,v=t*v,l=0))

35/2*v^2 + 35*v*w + 35/2*w^2 + 35/6

In [225]:
pretty(deg0(pretty(Verlinde_Num_quot_2_inv(5,1))(x=exp(t),y=exp(m),z=exp(l)))(w=t*w,v=t*v,l=0))

-63*v^2 - 126*v*w - 63*w^2 - 63/2

Maybe in general
$$-\frac{N}{2(N+1)}\binom{(N+1)n}{n} $$
for rank 1. 

For general rank, we have
$$-\frac{1}{2(r+1)}\binom{(r+1)n}{n} $$
for $N=1$. 

but for general rank and general $N$, maybe
$$-\frac{N}{2(N+r)}\binom{(N+r)n}{rn} $$
Checked for rank $1,2,-1,-2$, $N=1,2,3,4,5$. Note for negative ranks, lots of them become trivial due to vanishing from before.

$r=2,N=1$
$$\frac{1}{6}\binom{3n}{2n}$$

$r=2,N=2$
$$\frac{1}{4}\binom{4n}{2n}$$

$r=2,N=3$
$$\frac{3}{10}\binom{5n}{2n}$$

In [238]:
pretty(degN(pretty(Verlinde_Num_quot(3,3))(x=exp(t),y=exp(m),z=exp(l)),-1))

-55*(l + w)/t^2